In [1]:
import pandas as pd
import polars as pl
import re
import numpy as np
import os
import datetime as dt

In [2]:
RAW_DATA = '/work/InternalMedicine/s223850/ED-StaticDynamic/raw_data/ondemand-RWB1.csv'

In [3]:
df_rwb = pd.read_csv(RAW_DATA)

In [4]:
df_rwb

,Unnamed: 0,Race,Acuity,Acuity Abbr,Arrival Method,Arrival Time,CC,CSN,ED Disposition,Ethnicity,Patient Name/Age/Gender,Primary Dx,Primary ED Dx,Primary FC,Arrived
0,1,3,8.0,III,10.0,45900,BACK PAIN,706903249,DISCHARGED,13.0,"BANGE, A (61Y/O F)",ACUTE MIDLINE LOW BACK PAIN WITHOUT SCIATICA,ACUTE MIDLINE LOW BACK PAIN WITHOUT SCIATICA,2.0,5791092300
1,2,5,8.0,III,5.0,78416,CONSTIPATION,706909931,ADMITTED,3.0,"MALDONADO, M (59Y/O F)","CONSTIPATION, UNSPECIFIED CONSTIPATION TYPE","CONSTIPATION, UNSPECIFIED CONSTIPATION TYPE",101.0,5791038360
2,3,5,7.0,II,5.0,83100,RASH; OTHER,706910057,DISCHARGED,13.0,"VINES, C (73Y/O F)",RASH,RASH,2.0,5791043100
3,4,5,7.0,II,5.0,80760,DIFFICULTY BREATHING,706910107,DISCHARGED,3.0,"ARELLANO, S (24Y/O F)","CHEST PAIN, UNSPECIFIED TYPE","CHEST PAIN, UNSPECIFIED TYPE",101.0,5791040760
4,5,3,7.0,II,5.0,81100,GUN SHOT WOUND,706910138,TRANSFER TO ANOTHER FACILITY - CMC,13.0,"BUTLER, A (16Y/O M)",GSW (GUNSHOT WOUND),GSW (GUNSHOT WOUND),3.0,5791041060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,8,8.0,III,5.0,77460,BLADDER PAIN,713697694,NaN,3.0,"BONILLA, I (60Y/O M)",LOWER URINARY TRACT SYMPTOMS (LUTS),LOWER URINARY TRACT SYMPTOMS (LUTS),101.0,5799591060
9996,9997,8\n5,7.0,II,5.0,81180,CHEST PAIN,713698008,NaN,13.0,"DAMICO, A (77Y/O M)",LEFT ARM SWELLING,LEFT ARM SWELLING,2.0,5799594780
9997,9998,5,7.0,II,5.0,81360,BACK PAIN; DIFFICULTY BREATHING; CONFUSION,713698022,NaN,13.0,"JACKSON, L (95Y/O M)",ACUTE CONFUSIONAL STATE,ACUTE CONFUSIONAL STATE,2.0,5799594960
9998,9999,5,7.0,II,5.0,81660,OTHER,713698047,NaN,13.0,"KITCHELL, B (59Y/O M)",NaN,ESRD ON HEMODIALYSIS (*),101.0,5799595260


In [32]:
dx_len = df_rwb['Primary ED Dx'].apply(lambda x: x.split(';') if isinstance(x, str) else x).apply(lambda x: len(x) if isinstance(x, list) else 0)

In [28]:
cc_len = df_rwb['CC'].apply(lambda x: x.split(';') if isinstance(x, str) else x).apply(lambda x: len(x) if isinstance(x, list) else 0)

In [29]:
cc_len.min(), cc_len.max()

(0, 6)

In [33]:
dx_len.min(), dx_len.max()

(0, 3)

In [37]:
df_rwb.loc[dx_len==2, ['CC','Primary Dx', 'Primary ED Dx']]['Primary ED Dx'].iloc[-1]

'CHEST PAIN, UNSPECIFIED TYPE; HX COMPLETE HEART BLOCK W/ PPM'

In [26]:
df_rwb.loc[cc_len==2, ['CC', 'Primary Dx', 'Primary ED Dx']]

,CC,Primary Dx,Primary ED Dx
271,"WOUND, NON HEALING","OPEN WOUND OF RIGHT FOOT, INITIAL ENCOUNTER","OPEN WOUND OF RIGHT FOOT, INITIAL ENCOUNTER"
275,"WOUND, NON HEALING",CELLULITIS OF LEFT LEG,CELLULITIS OF LEFT LEG
560,"WOUND, NON HEALING","OPEN WOUND OF RIGHT GREAT TOE, INITIAL ENCOUNTER","OPEN WOUND OF RIGHT GREAT TOE, INITIAL ENCOUNTER"
893,"WOUND, NON HEALING",THERAPEUTIC DRUG MONITORING,WEAKNESS
1396,"HERNIA,INGUINAL",LEFT INGUINAL HERNIA,LEFT INGUINAL HERNIA
1411,"WOUND INFECTION, SURGICAL",SURGICAL SITE INFECTION,SURGICAL WOUND INFECTION
3400,"ABDOMINAL PAIN; HERNIA,UMBILICAL",ABDOMINAL HERNIA WITHOUT OBSTRUCTION AND WITHO...,ABDOMINAL HERNIA WITHOUT OBSTRUCTION AND WITHO...
3980,"WOUND INFECTION, SURGICAL",WOUND DISCHARGE,WOUND DISCHARGE
4131,"DIFFICULTY BREATHING; WOUND, NON HEALING",SHORTNESS OF BREATH,SHORTNESS OF BREATH
4389,"WOUND, NON HEALING","OPEN WOUND OF TOE, INITIAL ENCOUNTER","OPEN WOUND OF TOE, INITIAL ENCOUNTER"


In [4]:
pattern = r'\((\d+)Y/O\s([MF])\)'

In [5]:
df_rwb[['Patient_Age', 'Sex']] = df_rwb['Patient Name/Age/Gender'].str.extract(pattern)
df_rwb['Patient_Age'] = df_rwb['Patient_Age'].astype(float)

In [6]:
start_time = pd.Timestamp('1840-12-31 00:00:00')

In [7]:
df_rwb['Arrived_Time'] = start_time+pd.to_timedelta(df_rwb['Arrived'].astype(int), unit='s')

ValueError: invalid literal for int() with base 10: '5791375885[CDT'

In [ ]:
df_rwb['Arrived_Time']

In [164]:
df_rwb['Race'] = df_rwb['Race'].apply(lambda x: x.split('\n')[0] if isinstance(x, str) and len(x.split('\n'))>1 else x)

In [165]:
df_rwb = df_rwb.sort_values(by='Arrived_Time', ascending=False)


In [166]:
df_rwb.rename(columns={
    'CSN':'PAT_ENC_CSN_ID',
    'Unnamed: 0': 'index'
}, inplace=True)

In [167]:
df_rwb = df_rwb.drop(columns=['Acuity', 'Arrival Time', 'Patient Name/Age/Gender'])

In [83]:
# df_rwb['has_admit'] = 0
# df_rwb.loc[df_rwb['ED Disposition'].isnull(), 'has_admit'] = np.nan
# df_rwb.loc[df_rwb['ED Disposition']=='ADMITTED', 'has_admit'] = 1

In [84]:
# df_rwb['has_admit'].value_counts()

In [85]:
# df_rwb.drop(columns=['ED Disposition'], inplace=True)

# Make sure that every column has the same space of values of the data I have

In [2]:
df_o = pl.read_parquet('/work/InternalMedicine/s223850/ED-StaticDynamic/raw_data/ED Events Last 2 Years - compiled 6.6.24.parquet')

In [5]:
pats = np.random.choice(df_o['PAT_ENC_CSN_ID'].unique(), 10, replace=False)

In [8]:
dff = df_o.filter(pl.col("PAT_ENC_CSN_ID").is_in(pats))

In [10]:
dff.sort(by=['Arrived_Time', 'Calculated_DateTime']).write_csv('sample_original_data_10CSN.csv')

In [17]:
all_cc_list = df_o['Chief_Complaint_All'].to_pandas().str.split(',')

In [86]:
cc_orig = []
for d_list in all_cc_list:
    if d_list is None:
        continue
    cc_orig.extend(d_list)
    

In [87]:
all_cc_RWB = df_rwb['CC'].str.split(';')

In [88]:
cc_RWB = []
for d_list in all_cc_RWB:
    if d_list is None or isinstance(d_list, float):
        continue
    cc_RWB.extend(d_list)

In [89]:
cc_RWB = list(map(lambda x: x.strip(), cc_RWB))
cc_orig = list(map(lambda x: x.strip(), cc_orig))

In [90]:
cc_RWB_norm = list(map(lambda x: re.sub(f'[^a-zA-Z0-9_-]+', '', x), cc_RWB))
cc_orig_norm = list(map(lambda x: re.sub(f'[^a-zA-Z0-9_-]+', '', x), cc_orig))

In [91]:
cc_RWB_norm_uniq = np.unique(cc_RWB_norm)
cc_orig_norm_uniq = np.unique(cc_orig_norm)


In [92]:
included = 0
excluded = 0
for v in cc_RWB_norm_uniq:
    if v not in cc_orig_norm_uniq:
        print(v)
        excluded += 1
    else:
        included +=1

ARMSWELLINGREDNESS
BPELEVATED176103TINGLINGINBOTHARMSSOA27WEEKSPREGNANTBPELEVATED176103TINGLINGINBOTHARMSSOA27WEEKSPREGNANT
BUTTOCKPAIN
CTCSAFTERHOURSCALLCTCSAFTERHOURSCALL
DIFFICULTYBREATHING
GASTROESOPHOGEALREFLUX
HERNIAINGUINAL
HERNIAUMBILICAL
MEDICALQUESTIONPROBLEMPOST-PARTUMCOMPLICATIONSMEDICALQUESTIONPROBLEMPOST-PARTUMCOMPLICATIONSSHORTNESSOFAIRWITHANYEXERTIONINCREASEINEDEMAFROMTHEKNEESDOWNPRESSUREINHERCHESTPOSTOPERATIVEC-SECTION8212024
PAINFULMENSES
PERITONEALFLUIDINFECTION
RULEOUTPRETERMLABOR
SICKLECELL
URINARYTRACTINFECTION
WOUNDINFECTIONSURGICAL
WOUNDNONHEALING


In [95]:
included, excluded, len(cc_RWB_norm_uniq)

(287, 16, 303)

In [96]:
for v in cc_orig_norm_uniq:
    if 'sickle' in v.lower():
        print(v)

SICKLECELLCRISIS


In [97]:
for v in cc_orig_norm_uniq:
    if 'arms' in v.lower():
        print(v)


ARMSWELLING


In [98]:
for v in cc_orig_norm_uniq:
    if 'motor' in v.lower():
        print(v)


LOSSOFMOTORCONTROL
MOTORCYCLECRASH
MOTORVEHICLECRASH
MOTORVEHICLEVERSUSPEDESTRIAN


# Conclusion
Out of the 303 unique chief_complaints, there are 287 matches the CC I have in the original data.
Is there a ZC table for CC?

In [99]:
df_rwb.head()

,index,Race,Acuity Abbr,Arrival Method,CC,PAT_ENC_CSN_ID,ED Disposition,Ethnicity,Primary Dx,Primary ED Dx,Primary FC,Arrv Date/Time,Patient_Age,Sex,Arrived_Time
5208,5209,5,III,5.0,BACK PAIN,713267364,NaN,3.0,NaN,NaN,101.0,5.799170e+09,39.0,F,2024-10-07 00:29:00
5207,5208,3,II,5.0,SICKLE CELL,713267340,NaN,13.0,SICKLE CELL ANEMIA WITH PAIN (*),SICKLE CELL ANEMIA WITH PAIN (*),107.0,5.799170e+09,34.0,M,2024-10-07 00:25:00
5206,5207,NaN,III,5.0,SHOULDER INJURY,713267205,DISCHARGED,NaN,ACUTE PAIN OF LEFT SHOULDER,ACUTE PAIN OF LEFT SHOULDER,NaN,5.799168e+09,31.0,M,2024-10-07 00:08:00
5205,5206,5,III,5.0,ABDOMINAL PAIN,713267114,NaN,3.0,"ABDOMINAL PAIN, UNSPECIFIED ABDOMINAL LOCATION","ABDOMINAL PAIN, UNSPECIFIED ABDOMINAL LOCATION",NaN,5.799168e+09,29.0,M,2024-10-06 23:58:00
5204,5205,5,II,5.0,RECTAL PROBLEM,713267004,NaN,3.0,"DIARRHEA, UNSPECIFIED TYPE","DIARRHEA, UNSPECIFIED TYPE",2.0,5.799167e+09,64.0,M,2024-10-06 23:36:00


In [18]:
df_rwb.columns

Index(['index', 'Race', 'Acuity Abbr', 'Arrival Method', 'CC',
       'PAT_ENC_CSN_ID', 'ED Disposition', 'Ethnicity', 'Primary Dx',
       'Primary ED Dx', 'Primary FC', 'Arrv Date/Time', 'Patient_Age', 'Sex',
       'Arrived_Time'],
      dtype='object')

In [168]:
ZC_tables_dir = '/work/InternalMedicine/s223850/ED-StaticDynamic/raw_data/zc_tables/'
os.listdir(ZC_tables_dir)
for tab in os.listdir(ZC_tables_dir):
    df = pd.read_csv(os.path.join(ZC_tables_dir, tab))
    print(tab)

ClarityMirror_dbo_ZC_FINANCIAL_CLASS.csv
ClarityMirror_dbo_ZC_ACUITY_LEVEL.csv
ClarityMirror_dbo_ZC_PATIENT_RACE.csv
ClarityMirror_dbo_ZC_ETHNIC_GROUP.csv
ClarityMirror_dbo_ZC_ARRIV_MEANS.csv
ClarityMirror_dbo_ZC_SEX.csv


In [169]:
df_zc = pd.read_csv(os.path.join(ZC_tables_dir, 'ClarityMirror_dbo_ZC_ETHNIC_GROUP.csv'))

In [170]:
col_zc_map = {
    'Race':'ClarityMirror_dbo_ZC_PATIENT_RACE.csv',
    # 'Acuity Abbr':'ClarityMirror_dbo_ZC_ACUITY_LEVEL.csv',
    'Arrival Method': 'ClarityMirror_dbo_ZC_ARRIV_MEANS.csv',
    
    
}

In [171]:
race_dict = df_zc[['INTERNAL_ID', 'NAME']].set_index('INTERNAL_ID').to_dict()['NAME']

In [ ]:
df_rwb['Race'] = df_rwb['Race'].apply(lambda x: int(x) if x is not None and not isinstance(x, float) else -1).map(race_dict).fillna('unknown')

In [172]:
df_rwb['Race'] = df_rwb['Race'].apply(lambda x: int(x) if x is not None and not isinstance(x, float) else -1).map(race_dict).fillna('unknown')

In [173]:
df_zc_arrmean = pd.read_csv(os.path.join(ZC_tables_dir, 'ClarityMirror_dbo_ZC_ARRIV_MEANS.csv'))

In [174]:
arrmean_dict = df_zc_arrmean[['INTERNAL_ID', 'NAME']].set_index('INTERNAL_ID').to_dict()['NAME']

In [175]:
arrmean_dict

{1: 'Hospital Transport',
 10: 'Other',
 11: 'Referred by Clinic',
 12: 'Assist from Vehicle',
 13: 'Helicopter',
 14: 'Fixed Wing',
 2: 'Public Transportation',
 3: 'Police',
 4: 'Wheelchair',
 5: 'Car',
 6: 'Taxi',
 7: 'Medical Flight',
 8: 'Ambulance',
 9: 'Pedestrian'}

In [176]:
df_rwb['Arrival Method'] = df_rwb['Arrival Method'].apply(lambda x: int(x) if x is not None and not np.isnan(x) else -1).map(arrmean_dict).fillna('unknown')

In [177]:
df_zc_eth = pd.read_csv(os.path.join(ZC_tables_dir, 'ClarityMirror_dbo_ZC_ETHNIC_GROUP.csv'))
eth_dict = df_zc_eth[['INTERNAL_ID', 'NAME']].set_index('INTERNAL_ID').to_dict()['NAME']

In [178]:
df_rwb['Ethnicity'] = df_rwb['Ethnicity'].apply(lambda x: int(x) if x is not None and not np.isnan(x) else -1).map(eth_dict).fillna('unknown')

In [179]:
df_zc_fc = pd.read_csv(os.path.join(ZC_tables_dir, 'ClarityMirror_dbo_ZC_FINANCIAL_CLASS.csv'))
fc_dict = df_zc_fc[['INTERNAL_ID', 'NAME']].set_index('INTERNAL_ID').to_dict()['NAME']

In [180]:
df_rwb['Primary FC'] = df_rwb['Primary FC'].apply(lambda x: int(x) if x is not None and not np.isnan(x) else -1).map(fc_dict).fillna('unknown')

In [181]:
df_rwb

,index,Race,Acuity Abbr,Arrival Method,CC,PAT_ENC_CSN_ID,ED Disposition,Ethnicity,Primary Dx,Primary ED Dx,Primary FC,Arrived,Patient_Age,Sex,Arrived_Time
9999,10000,unknown,II,Car,RECTAL BLEEDING,713698278,NaN,Hispanic or Latino,RECTAL BLEEDING,RECTAL BLEEDING,Medicare,5799598680,81.0,M,2024-10-11 23:38:00
9998,9999,unknown,II,Car,OTHER,713698047,NaN,Non-Hispanic/Latino,NaN,ESRD ON HEMODIALYSIS (*),Managed Care,5799595260,59.0,M,2024-10-11 22:41:00
9997,9998,unknown,II,Car,BACK PAIN; DIFFICULTY BREATHING; CONFUSION,713698022,NaN,Non-Hispanic/Latino,ACUTE CONFUSIONAL STATE,ACUTE CONFUSIONAL STATE,Medicare,5799594960,95.0,M,2024-10-11 22:36:00
9996,9997,Unknown,II,Car,CHEST PAIN,713698008,NaN,Non-Hispanic/Latino,LEFT ARM SWELLING,LEFT ARM SWELLING,Medicare,5799594780,77.0,M,2024-10-11 22:33:00
9995,9996,Unknown,III,Car,BLADDER PAIN,713697694,NaN,Hispanic or Latino,LOWER URINARY TRACT SYMPTOMS (LUTS),LOWER URINARY TRACT SYMPTOMS (LUTS),Managed Care,5799591060,60.0,M,2024-10-11 21:31:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,unknown,III,Car,SYNCOPE,706910410,DISCHARGED,Hispanic or Latino,"SYNCOPE, UNSPECIFIED SYNCOPE TYPE",ORTHOSTATIC SYNCOPE,unknown,5791044180,18.0,F,2024-07-04 23:23:00
2,3,unknown,II,Car,RASH; OTHER,706910057,DISCHARGED,Non-Hispanic/Latino,RASH,RASH,Medicare,5791043100,73.0,F,2024-07-04 23:05:00
4,5,Hispanic or Latino,II,Car,GUN SHOT WOUND,706910138,TRANSFER TO ANOTHER FACILITY - CMC,Non-Hispanic/Latino,GSW (GUNSHOT WOUND),GSW (GUNSHOT WOUND),Medicaid,5791041060,16.0,M,2024-07-04 22:31:00
3,4,unknown,II,Car,DIFFICULTY BREATHING,706910107,DISCHARGED,Hispanic or Latino,"CHEST PAIN, UNSPECIFIED TYPE","CHEST PAIN, UNSPECIFIED TYPE",Managed Care,5791040760,24.0,F,2024-07-04 22:26:00


In [183]:
df_rwb = df_rwb.drop(columns=['Arrived']).set_index('index')

In [111]:
all_pdx = df_rwb['Primary Dx'].to_list() + df_rwb['Primary ED Dx'].to_list()

In [87]:
rwb_all_pdx_norm = list(map(lambda x: re.sub(r'[^a-zA-Z0-9,_-]+', '', x).lower() if isinstance(x, str) else x, all_pdx))

In [89]:
df_o = df_o.with_columns(
    pl.col("Primary_DX_Name").apply(
        lambda x: re.sub(r'[^a-zA-Z0-9,_-]+', '', x).lower() if x is not None else x
    ).alias("Primary_DX_Name_NORM")
)

/tmp/ipykernel_36888/1969497394.py:2: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col("Primary_DX_Name").apply(
/tmp/ipykernel_36888/1969497394.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df_o = df_o.with_columns(


In [90]:
df_o = df_o.with_columns(
    pl.col("Primary_DX_Name").str.replace_all(r'[^a-zA-Z0-9,_-]+', '')
    .str.to_lowercase()
    .alias("Primary_DX_NAME_NORM1")
)

In [98]:
all_pdx_o = df_o['Primary_DX_NAME_NORM1'].drop_nulls().to_list()

In [99]:
all_pdx_o_norm_uniq = np.unique(all_pdx_o)

In [100]:
all_pdx_rwb_norm_uniq = np.unique(rwb_all_pdx_norm)

In [104]:
included = 0
excluded = 0
for v in all_pdx_rwb_norm_uniq:
    if v not in all_pdx_o_norm_uniq:
        excluded += 1
    else:
        included += 1


print(included)
print(excluded)
print(len(all_pdx_rwb_norm_uniq))
x=0        

1512
350
1862


In [184]:

df_rwb.to_parquet('/work/InternalMedicine/s223850/ED-StaticDynamic/raw_data/rwb_static_clean_OCT92024.parquet')

# Combine the original static and RWB

In [4]:
df_rwb = pd.read_parquet('/work/InternalMedicine/s223850/ED-StaticDynamic/raw_data/rwb_static_clean_OCT92024.parquet')

In [5]:
df_o = pl.read_parquet('/work/InternalMedicine/s223850/ED-StaticDynamic/raw_data/ED Events Last 2 Years - compiled 6.6.24.parquet')

In [9]:
df_rwb = pl.from_dataframe(df_rwb)

In [6]:
df_rwb.columns

Index(['Race', 'Acuity Abbr', 'Arrival Method', 'CC', 'PAT_ENC_CSN_ID',
       'ED Disposition', 'Ethnicity', 'Primary Dx', 'Primary ED Dx',
       'Primary FC', 'Patient_Age', 'Sex', 'Arrived_Time'],
      dtype='object')

In [30]:
df_rwb = df_rwb.with_columns(
    pl.col('Race').alias('FirstRace'),
    pl.col('Acuity Abbr').alias('Acuity_Level'),
    pl.col('Arrival Method').alias('Means_Of_Arrival'),
    pl.col('CC').alias('Chief_Complaint_All'),
    pl.col('Primary FC').alias('Coverage_Financial_Class_Grouper'),
    pl.col('Primary Dx').alias('Primary_DX_Name'),
    pl.col('ED Disposition').alias('ED_Disposition')
)

In [31]:
df_os = df_o.select(['PAT_ENC_CSN_ID', 'FirstRace', 'Acuity_Level', 'Means_Of_Arrival', 'Arrived_Time', 'Chief_Complaint_All', 'Ethnicity',
             'Sex', 'Patient_Age', 'Coverage_Financial_Class_Grouper', 'Primary_DX_Name', 'ED_Disposition'])
             

In [33]:
df_agg_s = df_os.group_by('PAT_ENC_CSN_ID').agg(
    pl.col('FirstRace').last().alias('FirstRace'),
    pl.col('Acuity_Level').last().alias('Acuity_Level'),
    pl.col('Means_Of_Arrival').last().alias('Means_Of_Arrival'),
    pl.col('Arrived_Time').last().alias('Arrived_Time'),
    pl.col('Chief_Complaint_All').last().alias('Chief_Complaint_All'),
    pl.col('Ethnicity').last().alias('Ethnicity'),
    pl.col('Sex').last().alias('Sex'),
    pl.col('Patient_Age').last().alias('Patient_Age'),
    pl.col('Coverage_Financial_Class_Grouper').last().alias('Coverage_Financial_Class_Grouper'),
    pl.col('Primary_DX_Name').last().alias('Primary_DX_Name'),
    pl.col('ED_Disposition').last().alias('ED_Disposition')
)

In [25]:
df_rwb

Race,Acuity Abbr,Arrival Method,CC,PAT_ENC_CSN_ID,ED Disposition,Ethnicity,Primary Dx,Primary ED Dx,Primary FC,Patient_Age,Sex,Arrived_Time,FirstRace,Acuity_Level,Means_Of_Arrival,Chief_Complaint,Coverage_Financial_Class_Grouper,Primary_DX_Name,ED_Disposition,index,from_rwb,Chief_Complaint_All
str,str,str,str,i64,str,str,str,str,str,f64,str,datetime[ns],str,str,str,str,str,str,str,i64,i64,str
"""unknown""","""II""","""Car""","""RECTAL BLEEDIN…",713698278,null,"""Hispanic or La…","""RECTAL BLEEDIN…","""RECTAL BLEEDIN…","""Medicare""",81.0,"""M""",2024-10-11 23:38:00,"""unknown""","""II""","""Car""","""RECTAL BLEEDIN…","""Medicare""","""RECTAL BLEEDIN…",null,1,1,"""RECTAL BLEEDIN…"
"""unknown""","""II""","""Car""","""OTHER""",713698047,null,"""Non-Hispanic/L…",null,"""ESRD ON HEMODI…","""Managed Care""",59.0,"""M""",2024-10-11 22:41:00,"""unknown""","""II""","""Car""","""OTHER""","""Managed Care""",null,null,2,1,"""OTHER"""
"""unknown""","""II""","""Car""","""BACK PAIN; DIF…",713698022,null,"""Non-Hispanic/L…","""ACUTE CONFUSIO…","""ACUTE CONFUSIO…","""Medicare""",95.0,"""M""",2024-10-11 22:36:00,"""unknown""","""II""","""Car""","""BACK PAIN; DIF…","""Medicare""","""ACUTE CONFUSIO…",null,3,1,"""BACK PAIN; DIF…"
"""Unknown""","""II""","""Car""","""CHEST PAIN""",713698008,null,"""Non-Hispanic/L…","""LEFT ARM SWELL…","""LEFT ARM SWELL…","""Medicare""",77.0,"""M""",2024-10-11 22:33:00,"""Unknown""","""II""","""Car""","""CHEST PAIN""","""Medicare""","""LEFT ARM SWELL…",null,4,1,"""CHEST PAIN"""
"""Unknown""","""III""","""Car""","""BLADDER PAIN""",713697694,null,"""Hispanic or La…","""LOWER URINARY …","""LOWER URINARY …","""Managed Care""",60.0,"""M""",2024-10-11 21:31:00,"""Unknown""","""III""","""Car""","""BLADDER PAIN""","""Managed Care""","""LOWER URINARY …",null,5,1,"""BLADDER PAIN"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""unknown""","""III""","""Car""","""SYNCOPE""",706910410,"""DISCHARGED""","""Hispanic or La…","""SYNCOPE, UNSPE…","""ORTHOSTATIC SY…","""unknown""",18.0,"""F""",2024-07-04 23:23:00,"""unknown""","""III""","""Car""","""SYNCOPE""","""unknown""","""SYNCOPE, UNSPE…","""DISCHARGED""",9996,1,"""SYNCOPE"""
"""unknown""","""II""","""Car""","""RASH; OTHER""",706910057,"""DISCHARGED""","""Non-Hispanic/L…","""RASH""","""RASH""","""Medicare""",73.0,"""F""",2024-07-04 23:05:00,"""unknown""","""II""","""Car""","""RASH; OTHER""","""Medicare""","""RASH""","""DISCHARGED""",9997,1,"""RASH; OTHER"""
"""Hispanic or La…","""II""","""Car""","""GUN SHOT WOUND…",706910138,"""TRANSFER TO AN…","""Non-Hispanic/L…","""GSW (GUNSHOT W…","""GSW (GUNSHOT W…","""Medicaid""",16.0,"""M""",2024-07-04 22:31:00,"""Hispanic or La…","""II""","""Car""","""GUN SHOT WOUND…","""Medicaid""","""GSW (GUNSHOT W…","""TRANSFER TO AN…",9998,1,"""GUN SHOT WOUND…"


In [26]:
df_rwb = df_rwb.with_columns([pl.arange(1, len(df_rwb)+1,1).alias('index'), pl.lit(1).cast(pl.Int64).alias('from_rwb')])


In [34]:
df_rwb_s = df_rwb.select(['index', 'PAT_ENC_CSN_ID', 'FirstRace', 'Acuity_Level', 'Means_Of_Arrival', 'Arrived_Time', 'Chief_Complaint_All', 'Ethnicity', 'Sex', 'Patient_Age', 'Coverage_Financial_Class_Grouper', 'Primary_DX_Name', 'ED_Disposition', 'from_rwb'])

In [36]:
df_agg_s = df_agg_s.with_columns([pl.arange(1, len(df_agg_s)+1,1).alias('index'), pl.lit(0).cast(pl.Int64).alias('from_rwb')])
df_agg_s = df_agg_s.with_columns(
    pl.col('Arrived_Time').str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S%.f"),
)

In [37]:
df_comb = pl.concat([df_agg_s, df_rwb_s.select(df_agg_s.columns)], how='vertical')

In [38]:
df_comb.filter(pl.col('Arrived_Time')>=dt.datetime(2024, 4, 1)).write_parquet('/work/InternalMedicine/s223850/ED-StaticDynamic/raw_data/orig_rwb_comb_April2Oct2024.parquet')

In [65]:
df_rwb_s.schema


OrderedDict([('index', Int64),
             ('PAT_ENC_CSN_ID', Int64),
             ('Race', String),
             ('Acuity Abbr', String),
             ('Arrival Method', String),
             ('Arrived_Time', Datetime(time_unit='ns', time_zone=None)),
             ('CC', String),
             ('Ethnicity', String),
             ('Sex', String),
             ('Patient_Age', Float64),
             ('Primary FC', String),
             ('Primary Dx', String),
             ('ED Disposition', String),
             ('from_rwb', Int64)])

In [67]:
df_agg_s.schema

OrderedDict([('PAT_ENC_CSN_ID', Int64),
             ('Race', String),
             ('Acuity Abbr', String),
             ('Arrival Method', String),
             ('Arrived_Time', Datetime(time_unit='ns', time_zone=None)),
             ('CC', String),
             ('Ethnicity', String),
             ('Sex', String),
             ('Patient_Age', Float64),
             ('Primary FC', String),
             ('Primary Dx', String),
             ('ED Disposition', String),
             ('index', Int64),
             ('from_rwb', Int32)])